# CHAMP Transects Notebook

<img style="float:right;" src="https://id.dewberry.com/clients/Templates/DewberryLogo_RGB.png" width=150px>

### Description
Read data from Microsoft Access

---

#### - Requirements
  1. whafis_champ library
  2. Microsoft Access files
  3. pyodc python library
  
#### - Usage:
  1. Import Libraries & Assign Paths
  2. Read in data from files:
  
 ---

### Import Libraries

In [1]:
import pyodbc
import pandas as pd
import numpy as np
from importlib import reload
import whafis_champ ; reload(whafis_champ)
from whafis_champ import *
from matplotlib import pyplot as plt
import os
%matplotlib inline

### Assign Paths

In [2]:
root_dir = r'P:\Temp\tmiesse\for_Jeff_transects'
mbd      = 'Charlotte_CHAMP.mdb'
path     = os.path.join(root_dir,mbd)
xl1, xl2, xl25, xl3, xl4 = [],[],[],[],[]
yl1, yl2, yl25, yl3, yl4 = [],[],[],[],[]

### Open the Microsoft Access Data

#### EROSION

In [3]:
select = 'EROSION'
dfE = micro_access_open(path,select)
dfE.rename(columns={0:'Transect', 1:'Station', 2:'Elevation'}, inplace=True)
transects = list(dfE['Transect'].unique() )

#### Geometry

In [4]:
select = 'GEOMETRY'
dfG = micro_access_open(path,select)
dfG.rename(columns={0:'Transect', 1:'SWFL1', 2:'Approach_slope', 3:'Face_Slope', 4:'Remove_slope',
                    5:'Seaward Slope', 6:'Critical Area', 7:'Erosion Status', 8:'Reservoir Area'}, inplace=True)
transects_geo = list(dfG['Transect'].unique() )

#### TRANSECT

In [5]:
select = 'TRANSECT'
dfT2 = micro_access_open(path,select)
dfT2.rename(columns={0:'Transect', 1:'Station', 2:'Elevation'}, inplace=True)
transects_orig = list(dfT2['Transect'].unique())

#### WHAFIS Part 2

In [6]:
select = '[WHAFIS PART 2]'
dfW2 = micro_access_open(path,select)
dfW2.rename(columns={0:'Transect', 1:'WHAFIS_CARD', 2:'Stations',3:'Wave_height',
                     4:'Wave_period',5:'Wave_crest_elevation'}, inplace=True)
transects_wave = list(dfW2['Transect'].unique())

#### STILLWATER

In [7]:
select = 'STILLWATER'
dfS2 = micro_access_open(path,select)
dfS2.rename(columns={0:'Transect', 1:'Station', 2:'10_year_swell',3:'100_year_swell'}, inplace=True)
transects_still = list(dfS2['Transect'].unique())

#### WHAFIS PART 6

In [8]:
select = '[WHAFIS PART 6]'
dfZ2 = micro_access_open(path,select)
dfZ2.rename(columns={0:'Transect', 1:'Station begin', 2:'elevation begin',3:'station end',4:'elevation end',5:'zone',
                    6:'zone elevation',7:'FHF'}, inplace=True)
transects_zone = list(dfZ2['Transect'].unique())

### Plot Data

In [10]:
title = 'Charlotte County - Wave Envelope Profile'

In [ ]:
for t in transects[1:]:
    erosion_profile = dfE[dfE['Transect'] == t]
    sorted_erosion = erosion_profile.sort_values(by='Station')
    original_profile = dfT2[dfT2['Transect'] == t]
    sorted_original = original_profile.sort_values(by='Station')
    zone_profile = dfZ2[dfZ2['Transect'] == t]
    sorted_zone = zone_profile.sort_values(by='Station begin')
    geo_profile = dfG[dfG['Transect'] == t]
    change = pd.merge(sorted_original,sorted_erosion,how='outer',indicator='Exist')
    change = change.loc[change['Exist'] != 'both']
    if t in transects_wave:
        if len(change)>1:
            fig = plt.figure(figsize=(18,10))
            ax1 = plt.axes([0,0.7,1,0.3])
            ax2 = plt.axes([0,0,1,0.6])
            ax1.plot(sorted_erosion['Station'], sorted_erosion['Elevation'],'k')
            ax1.plot(sorted_original['Station'],sorted_original['Elevation'],'--k')
            ax2.plot(sorted_original['Station'],sorted_original['Elevation'],'k')
            change2 = change.reset_index(drop=True).copy()
            ax1.scatter(change2['Station'].iloc[-1],change2['Elevation'].iloc[-1],s=100,c='m')
            wave_profile = dfW2[dfW2['Transect'] == t]
            sorted_wave = wave_profile.sort_values(by='Stations')
            ax2.plot(sorted_wave['Stations'], sorted_wave['Wave_crest_elevation'],'-m')
            ax2.plot(sorted_wave['Stations'], sorted_wave['Wave_height'],'-c')
            still_profile = dfS2[dfS2['Transect'] == t]
            sorted_still = still_profile.sort_values(by='Station')
            ax2.plot(sorted_still['Station'], sorted_still['100_year_swell'],'-b')
            sorted_wave = sorted_wave.reset_index(drop=True).copy()
            geo_name = geo_profile.reset_index(drop=True).copy()
            wv_len = np.arange(sorted_wave['Stations'][0],sorted_wave['Stations'].iloc[-1],1)
            wave_h = np.interp(wv_len,list(sorted_wave['Stations']),list(sorted_wave['Wave_height']))
            for i in range(0,len(wave_h)):
                if wave_h[i] >= 3:
                    xl1.append(wv_len[i])
                    yl1.append(wv_len[i]*0+2.75*sorted_erosion['Elevation'].max())
                    
                if wave_h[i] < 3 and wave_h[i] >= 1.5:
                    xl2.append(wv_len[i])
                    yl2.append(wv_len[i]*0+2.75*sorted_erosion['Elevation'].max())
                    
                if wave_h[i] < 1.5 and wave_h[i] >=0.5:
                    xl3.append(wv_len[i])
                    yl3.append(wv_len[i]*0+2.75*sorted_erosion['Elevation'].max())
                    
                if wave_h[i] < 0.5:
                    xl4.append(wv_len[i])
                    yl4.append(wv_len[i]*0+2.75*sorted_erosion['Elevation'].max())
                    
            ax2.plot(xl1,yl1,'og',linewidth=4.0)
            ax2.plot(xl2,yl2,'or',linewidth=4.0)
            ax2.scatter(xl3,yl3,c='#247afd',marker='o',s=40.0)
            ax2.scatter(xl4,yl4,c='grey',marker='o',s=40.0)
            xl1, xl2, xl25, xl3, xl4 = [],[],[],[],[]
            yl1, yl2, yl25, yl3, yl4 = [],[],[],[],[]

            ax2.legend(['*Terrain','Wave Crest','**Controlling Wave Height','1% Total SWEL','VE Zone',
                        'AE Zone','A Zone','X Zone'],loc='upper center',bbox_to_anchor=(0.5,-0.15),ncol=7)
            ax1.legend(['Terrain','Pre-Eroded Terrain','PFD'],loc='upper center',bbox_to_anchor=(0.5,-0.15),ncol=7)
            ax1.set_xlim(0.5*change['Station'].min(), 1.25*change['Station'].max())
            ax1.set_ylim(0.25*change['Elevation'].min(), 1.25*change['Elevation'].max())
            ax1.set_xlabel('Station (feet)')
            ax1.set_ylabel('Elevation (feet NAVD88)')
            ax1.grid(True)
            ax1.set_title(title+': at Transect {}'.format(t) + '\n' + 
                          'Erosion Status {}'.format(geo_name['Erosion Status'][0])+
                          '\n'+ 'Revervoir Area = {} ft^2'.format(geo_name['Reservoir Area'][0]))
            ax2.set_xlim(-50, sorted_erosion['Station'].max())
            ax2.set_ylim(1.2*sorted_erosion['Elevation'].min(), 6.5*sorted_erosion['Elevation'].max())
            ax2.set_xlabel('Station (feet)')
            ax2.set_ylabel('Elevation (feet NAVD88)')
            ax2.grid(True)

        else:
            fig, ax = plt.subplots(figsize=(18,10))
            z1=ax.plot(sorted_original['Station'],sorted_original['Elevation'],'k')
            wave_profile = dfW2[dfW2['Transect'] == t]
            sorted_wave = wave_profile.sort_values(by='Stations')
            z2=ax.plot(sorted_wave['Stations'], sorted_wave['Wave_crest_elevation'],'-m')
            z3=ax.plot(sorted_wave['Stations'], sorted_wave['Wave_height'],'-c')
            still_profile = dfS2[dfS2['Transect'] == t]
            sorted_still = still_profile.sort_values(by='Station')
            z4=ax.plot(sorted_still['Station'], sorted_still['100_year_swell'],'-b')
            sorted_wave = sorted_wave.reset_index(drop=True).copy()
            wv_len = np.arange(sorted_wave['Stations'][0],sorted_wave['Stations'].iloc[-1],1)
            wave_h = np.interp(wv_len,list(sorted_wave['Stations']),list(sorted_wave['Wave_height']))
            for i in range(0,len(wave_h)):
                if wave_h[i] >= 3:
                    xl1.append(wv_len[i])
                    yl1.append(wv_len[i]*0+2.75*sorted_erosion['Elevation'].max())
                    
                if wave_h[i] < 3 and wave_h[i] >= 1.5:
                    xl2.append(wv_len[i])
                    yl2.append(wv_len[i]*0+2.75*sorted_erosion['Elevation'].max())
                    
                if wave_h[i] < 1.5 and wave_h[i] >=0.5:
                    xl3.append(wv_len[i])
                    yl3.append(wv_len[i]*0+2.75*sorted_erosion['Elevation'].max())
                    
                if wave_h[i] < 0.5:
                    xl4.append(wv_len[i])
                    yl4.append(wv_len[i]*0+2.75*sorted_erosion['Elevation'].max())
                    
            ax.plot(xl1,yl1,'og',linewidth=4.0)
            ax.plot(xl2,yl2,'or',linewidth=4.0)
            ax.scatter(xl3,yl3,c='#247afd',marker='o',s=40.0)
            ax.scatter(xl4,yl4,c='grey',marker='o',s=40.0)
            xl1, xl2, xl25, xl3, xl4 = [],[],[],[],[]
            yl1, yl2, yl25, yl3, yl4 = [],[],[],[],[]
    
            ax.legend(['*Terrain','Wave Crest','**Controlling Wave Height','1% Total SWEL','VE Zone',
                       'AE Zone','A Zone','X Zone'],loc='upper center',bbox_to_anchor=(0.5,-0.15),ncol=7)
                        
            ax.set_xlim(-50, sorted_erosion['Station'].max())
            ax.set_ylim(1.2*sorted_erosion['Elevation'].min(), 6.5*sorted_erosion['Elevation'].max())
            ax.set_xlabel('Station (feet)')
            ax.set_ylabel('Elevation (feet NAVD88)')
            ax.set_title(title+': at Transect {}'.format(t))
            ax.grid(True)

    plt.savefig(root_dir+'\\'+mbd.split('.mdb')[0]+'_transect_{}.png'.format(t),dpi=300, bbox_inches = 'tight', pad_inches = 0.1)
    #plt.show()
    plt.close() 